In [ ]:
%pip install langchain_groq
%pip install langchain_core langchain_community
%pip install pypdf
%pip install chromadb
%pip install sentence_transformers

In [ ]:
%pip uninstall -y keras tensorflow
%pip install tensorflow==2.15.0 tf-keras

In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq (
    temperature =0,
    groq_api_key ="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", 
    model_name ="llama-3.3-70b-versatile"
)

result = llm.invoke("how do u feel after listening to word Anmol")
print(result.content)

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os 

def initialize_llm():
    llm = ChatGroq (
    temperature =0,
    groq_api_key ="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", 
    model_name ="llama-3.3-70b-versatile"
)
    return llm


def create_vector_db():
    loader = DirectoryLoader("/content/", glob="*.pdf", loader_cls= PyPDFLoader)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma.from_documents(texts, embeddings, persist_directory='./chroma_db')
    vector_db.persist()

    print("chroma db created and data saved")

    return vector_db

def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt_templates="""  

    mental health chatbot 
    {context}
    User : {question}    
    Chatbot:"""
    
    PROMPT = PromptTemplate(template = prompt_templates, input_variables=['context', 'question'])
    qa_chain = RetrievalQA.from_chain_type(
        llm = llm,
        chain_type="stuff",
        retriever = retriever,
        chain_type_kwargs = {"prompt": PROMPT}
    )
    return qa_chain

  
def main():
  print("chatbot started")
  llm = initialize_llm()

  db_path = "/content/chroma_db"

  if not os.path.exists(db_path):
    vector_db = create_vector_db()
  else:
    embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
  qa_chain = setup_qa_chain(vector_db, llm)

  while True:
    query = input("\nHuman: ")
    if query.lower() == "exit":
      print("ChatBot: Take Care")
      break
    response = qa_chain.run(query)
    print(f"Chatbot: {response}")

if __name__ == "__main__":
  main()


In [ ]:
create_vector_db()